In [ ]:
import torch 
from torch import nn 

# Torchvision
import torchvision 
from torchvision import datasets 
from torchvision.transforms import ToTensor # Convert a PIL Image or ndarray to tensor and scale the values accordingly.

import matplotlib.pyplot as plt

# Checking versions

print(F"PyTorch versrion : {torch.__version__}")
print(F"TorchVision version : {torchvision.__version__}")

In [ ]:
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(), # images come as PIL format, we want to turn into Torch tensors
    target_transform=None, 
)

# Testing Data 
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
image, label = train_data[0]
print(F"image: {image} \n \n label : {label}")

In [ ]:
# Input and Output shapes of data 
image.shape

In [ ]:
# Data Shape 

len(train_data.data), len(train_data.targets), len(test_data.data), len(test_data.targets)

In [ ]:
class_names  = train_data.classes
print(class_names)

In [ ]:
import matplotlib.pyplot as plt
image, label = train_data[0]
print(f"Image shape: {image.shape}")
plt.imshow(image.squeeze()) # image shape is [1, 28, 28] (colour channels, height, width)
plt.title(class_names[label]);

In [ ]:
# GreyScale 

plt.imshow(image.squeeze(), cmap="gray")
plt.title(class_names[label]);

In [ ]:
# Plot more images
torch.manual_seed(42)
fig = plt.figure(figsize=(9, 9))
rows, cols = 4, 4
for i in range(1, rows * cols + 1):
    random_idx = torch.randint(0, len(train_data), size=[1]).item()
    img, label = train_data[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.squeeze(), cmap="gray")
    plt.title(class_names[label])
    plt.axis(False);

In [ ]:
# Crearting DataLoader 

from torch.utils.data import DataLoader

# Batch-size Hyperparameter 

BATCH_SIZE = 32

# turining data into iterables ( Batches )

train_dataloader = DataLoader(
    dataset=train_data, # dataset to be turned into an iterable 
    batch_size=BATCH_SIZE, # How many samples per batch 
    shuffle=True, # shuffle data after every epoch ?
)

test_dataloader = DataLoader(
    dataset=test_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}") 
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")

In [ ]:
# What is inside training Dataloader 

train_features_batch, train_labels_batch = next(iter(train_dataloader))
# is used to extract a batch of data from a PyTorch dataloader.

In [ ]:
# Show a sample
torch.manual_seed(42)
random_idx = torch.randint(0, len(train_features_batch), size=[1]).item() 
                    # In PyTorch, the item() method is used to retrieve 
                    # the value of a tensor as a standard Python scalar. 
                    # It is typically used when you have a tensor with a single element, 
                    # such as a tensor representing a loss value or a single prediction.

img, label = train_features_batch[random_idx], train_labels_batch[random_idx]
plt.imshow(img.squeeze(), cmap="gray")
plt.title(class_names[label])
plt.axis("Off");
print(f"Image size: {img.shape}")
print(f"Label: {label}, label size: {label.shape}")

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
# Flatening the input

# nn.Flatten() compresses the dimensions of a tensor into a single vector.

flatten_model = nn.Flatten()    # all nn modules function as a model, 
                                # ie can do a forward pass 
# Getting a single sample 

x = train_features_batch[0]

# Flattening the sample 
output = flatten_model(x) # forward pass 

# Printing output 

print(f"Shape before flattening: {x.shape} -> [color_channels, height, width]")
print(f"Shape after flattening: {output.shape} -> [color_channels, height*width]")

In [ ]:
# Model 0
# Simple Linear Model 


from torch import nn
from torch.nn.modules.linear import Linear

class FashionMNISTModelV0(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=input_shape, out_features=hidden_units), # in_features = number of features in a data sample (784 pixels)
            nn.Linear(in_features=hidden_units, out_features=output_shape),
        )

    def forward(self, x):
        return self.layer_stack(x)

In [ ]:
# Instantiate the Model 

model_0 = FashionMNISTModelV0(
    input_shape=28*28,
    hidden_units=10,
    output_shape=len(class_names),
).to("cpu")

print(model_0)

In [ ]:
# Importing from previous modules 

import requests
from pathlib import Path 

# Download helper functions from Learn PyTorch repo (if not already downloaded)
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  # Note: you need the "raw" GitHub URL for this to work
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

In [ ]:
# Backup accuracy 
from helper_functions import accuracy_fn

In [ ]:
# Accuracy 

try:
    from torchmetrics import Accuracy 
except:
    !pip install torchmetrics
    from torchmetrics import Accuracy

accuracy = Accuracy(task="multiclass",
                    num_classes=len(class_names),
                    )

print(accuracy)

In [ ]:
# Loss Function
loss_fn = nn.CrossEntropyLoss() # This criterion computes the cross entropy loss
                                # between input logits and target.

print(loss_fn)

In [ ]:
# Optimizer 
optimizer = torch.optim.SGD(
    params=model_0.parameters(),
    lr=0.1
)
print(optimizer)

In [ ]:
from timeit import default_timer as timer 
def print_train_time(start: float, end: float, device: torch.device = None):
    """Prints difference between start and end time.

    Args:
        start (float): Start time of computation (preferred in timeit format). 
        end (float): End time of computation.
        device ([type], optional): Device that compute is running on. Defaults to None.

    Returns:
        float: time between start and end in seconds (higher is longer).
    """
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time

In [ ]:
# for progress bar
from tqdm.auto import trange, tqdm 

# Training Loop 
# Start seed 

torch.manual_seed(42)
train_time_start_on_cpu = timer()

# Number of Epochs 
epochs = 10

# Training and testing loop

for epoch in tqdm(range(epochs)):
    print(F"Epoch: {epoch}\n------")

    # Training 
    train_loss = 0

    # loop to loop through the batches 
    for batch, (X,y) in tqdm(enumerate(train_dataloader)):    # enumerate() is a built-in function in Python 
                                                        # that allows you to iterate over a sequence 
                                                        # while keeping track of the index of each element. 
                                                        # It takes an iterable (such as a list, tuple, or string) 
                                                        # as input and returns an iterator that generates 
                                                        # pairs of index and corresponding elements.
        model_0.train()

        # 1. Forward Pass 
        y_pred = model_0(X) # output logits  

        # 2. Calculate the loss 
        loss = loss_fn(y_pred, y) # loss for this specific batch
                                  # input logits 
        train_loss += loss # accumulate loss per epoch 

        # 3. Optimizer Zero grad
        optimizer.zero_grad()

        # 4. Loss baclward 
        loss.backward() # Backprop adone per batch 

        # 5.Optimizer step
        optimizer.step()

        # print out how many batches sample have seen
        if batch%400 == 0:
            print(F"Looked at {batch*len(X)}/{len(train_dataloader.dataset)} sampples")

    # Average loss per batch, per epoch 
    # Divide total train loss by length of train dataloader 

    train_loss /= len(train_dataloader)

    # Testing 

    test_loss, test_acc = 0,0
    model_0.eval()

    with torch.inference_mode():
        for X,y in tqdm(test_dataloader):
            
            # 1. Forward Pass
            test_pred = model_0(X)

            # 2. Calculate loss ( accumutively )
            test_loss += loss_fn(test_pred, y)

            # 3. Calculate Accuracy 
                                    # ( preds need to be same as y_true 
                                    # if using accuracy_fn from helper file
                                    # if using torchmetrics accuracy, 
                                    # from docs : 
                                    # preds (Tensor): An int tensor of shape (N, ...) or 
                                    # float tensor of shape (N, C, ..). 
                                    # If preds is a floating point we apply torch.argmax 
                                    # along the C dimension to automatically convert probabilities/logits into an int tensor. )
                                    # target (Tensor): An int tensor of shape (N, ...)

            test_acc += accuracy(preds=test_pred, target=y )
            #test_acc += accuracy(preds=test_pred.argmax(dim=1), target=y ) # this should work too 
            #test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1)) # jsut to test output accuracy format in %
                                                                                # Found the bug
                                                                                # if accuracy_fn be used, the accuracy is being multiplided by 
                                                                                # a factor of 100 in origical code implementation
        # Calculations on test metrics need to happen inside torch.inference_mode()
        # Divide total test loss by length of test dataloader (per batch)
        test_loss /= len(test_dataloader)

        # Divide total accuracy by length of the test dataloader ( per batch )
        test_acc /= len(test_dataloader)

    ## print out whats happening 
    print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f}, Test acc: {test_acc:.4f}\n")
    
# Calculate Training time 

train_time_end_on_cpu = timer()
total_train_time_model_0 = print_train_time(start=train_time_start_on_cpu, 
                                           end=train_time_end_on_cpu,
                                           device=str(next(model_0.parameters()).device))

In [ ]:
## Model Evaluation
torch.manual_seed(42)
# make device agnostic

def eval_model(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy,
               device: torch.device = device):
    """Returns a dictionary containing the results of model predicting on data_loader.

    Args:
        model (torch.nn.Module): A PyTorch model capable of making predictions on data_loader.
        data_loader (torch.utils.data.DataLoader): The target dataset to predict on.
        loss_fn (torch.nn.Module): The loss function of model.
        accuracy_fn: An accuracy function to compare the models predictions to the truth labels.
        device (str, optional): Target device to compute on. Defaults to device.
    Returns:
        (dict): Results of model making predictions on data_loader.
    """
    loss, acc = 0, 0
    model.to(device)
    model.eval()
    with torch.inference_mode():
        for X,y in data_loader: # using 32 images as a batch
            
            # Send data to the target device
            X, y = X.to(device), y.to(device)
            # Making predictions
            y_pred = model(X)

            # Accumulate the loss and accuracy values per batch
            loss += loss_fn(y_pred, y)
            accuracy.to(device) 
            acc += accuracy(preds=y_pred, target=y)
        
        # Scale loss and acc to find the average loss/acc per batch
        loss /= len(data_loader)
        acc /= len(data_loader) 

    return {"model_name": model.__class__.__name__ , # only works when model was created with a class
            "model_loss": loss.item(),
            "model_acc" : acc.item(), 
            } 
print(eval_model)

In [ ]:
# Evaluating Model 0 
model_0_results = eval_model(model=model_0,
                             data_loader=test_dataloader,
                             loss_fn=loss_fn,
                             accuracy=accuracy)
print(model_0_results)

In [ ]:
# Non-Linear model with ReLU 

import torch
from torch import nn

class FashionMNISTModelV1(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()

        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=input_shape, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_shape),
            nn.ReLU(),
        )
    def forward(self, x: torch.Tensor):
        return self.layer_stack(x)

In [ ]:
# Instantiating the Model 

torch.manual_seed(42)
model_1 = FashionMNISTModelV1(input_shape=28*28, # number of input features
    hidden_units=10,
    output_shape=len(class_names) # number of output classes desired
).to(device) # send model to GPU if it's available
next(model_1.parameters()).device # check model device

In [ ]:
# Accuracy 

try:
    from torchmetrics import Accuracy 
except:
    !pip install torchmetrics
    from torchmetrics import Accuracy

accuracy = Accuracy(task="multiclass",
                    num_classes=len(class_names),
                    )

print(accuracy)

# Loss
loss_fn = nn.CrossEntropyLoss()
print(loss_fn)

# Optimizer 

optimizer = torch.optim.SGD(params=model_1.parameters(), lr=0.1)
print(optimizer)

In [ ]:
# Training Step
from tqdm.auto import tqdm 

def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy,
               device: torch.device = device,
               ):
    
    train_loss, train_acc = 0, 0
    model.to(device)
    model.train()

    for batch, (X, y) in tqdm(enumerate(data_loader)):
        
        # Sending data to GPU
        X,y = X.to(device), y.to(device)

        # 1. Forward Pass
        y_pred = model(X) # calculate logits 

        # 2. Calculate the loss 
        loss = loss_fn(y_pred, y) # loss for this specific batch
                                  # input logits 
        train_loss += loss # accumulate loss per epoch
        accuracy.to(device) 
        train_acc += accuracy(target=y, preds=y_pred)

        # 3. Optimizer Zero grad
        optimizer.zero_grad()

        # 4. Loss baclward 
        loss.backward() # Backprop adone per batch 

        # 5.Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.3f}")

In [ ]:
# Testing Step 
from tqdm.auto import tqdm

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy,
              device: torch.device = device,
              ):
    
    test_loss, test_acc = 0, 0
    model.to(device)
    model.eval() # put model in eval mode

    
    with torch.inference_mode(): # Turn on inference context manager
        for X, y in tqdm(data_loader):

            # Send data to GPU
            X, y = X.to(device), y.to(device)
            
            # 1. Forward pass
            test_pred = model(X)
            
            # 2. Calculate loss and accuracy
            
            test_loss += loss_fn(test_pred, y)
            accuracy.to(device) 
            test_acc += accuracy(target=y, preds=test_pred)
        
        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:3f}\n")

In [ ]:
torch.manual_seed(42)

# Measure time

from timeit import default_timer as timer 
train_time_start_on_gpu = timer()
from tqdm.auto import tqdm

epochs = 10

for epoch in tqdm(range(epochs)):
    print(F"Epoch: {epoch} \n------------")
    train_step(data_loader=train_dataloader,
               model=model_1,
               loss_fn=loss_fn,
               optimizer=optimizer,
               accuracy=accuracy,
               )
    test_step(data_loader=test_dataloader,
        model=model_1,
        loss_fn=loss_fn,
        accuracy=accuracy,
    )

train_time_end_on_gpu = timer()
total_train_time_model_1 = print_train_time(start=train_time_start_on_gpu,
                                            end=train_time_end_on_gpu,
                                            device=device)

In [ ]:
model_1_results = eval_model(model=model_1,
                             data_loader=test_dataloader,
                             loss_fn=loss_fn,
                             accuracy=accuracy,
                             device=device,)
print(model_1_results)

In [ ]:
# Baseline results : 
print(model_0_results)

In [ ]:
# Creating a Convolutional Neural Network
# Network Architecture from TinyVGG

import torch
from torch import nn 

class FashionMNISTModelV2(nn.Module):

    def __init__(self,input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()

        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1, #default
                      padding=1), # options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number 
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride value is same as kernel_size
                                    )
        
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units,hidden_units,3,1,1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
                                     )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*7*7,
                      out_features=output_shape)
                                        )
        
    def forward(self,x: torch.Tensor):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.classifier(x)
        
        return x

In [ ]:
torch.manual_seed(42)
model_2 = FashionMNISTModelV2(input_shape=1, 
    hidden_units=10, 
    output_shape=len(class_names)).to(device)
model_2

In [ ]:
torch.manual_seed(42)

# Create sample batch of random numbers with same size as image batch
images = torch.randn(size=(32, 3, 64, 64)) # [batch_size, color_channels, height, width]
test_image = images[0] # get a single image for testing
print(f"Image batch shape: {images.shape} -> [batch_size, color_channels, height, width]")
print(f"Single image shape: {test_image.shape} -> [color_channels, height, width]") 
print(f"Single image pixel values:\n{test_image}")

In [ ]:
# toy Conv2d

torch.manual_seed(42)

# Create a convolutional layer with same dimensions as TinyVGG 
# (try changing any of the parameters and see what happens)
conv_layer = nn.Conv2d(in_channels=3,
                       out_channels=10,
                       kernel_size=3,
                       stride=1,
                       padding=0)   # also try using "valid" or "same" here 
                                    # padding=0 and padding="valid" both mean no padding is applied, 
                                    # while padding="same" adds padding symmetrically 
                                    # to preserve the spatial dimensions of the input. 

                                    

# Pass the data through the convolutional layer
conv_layer(test_image)  # Note: If running PyTorch <1.11.0, 
                        # this will error because of shape issues 
                        #(nn.Conv.2d() expects a 4d tensor as input) 

In [ ]:
# Add extra dimension to test image
test_image.unsqueeze(dim=0).shape


In [ ]:
# Pass test image with extra dimension through conv_layer
conv_layer(test_image.unsqueeze(dim=0)).shape

In [ ]:
torch.manual_seed(42)
# Create a new conv_layer with different values (try setting these to whatever you like)
conv_layer_2 = nn.Conv2d(in_channels=3, # same number of color channels as our input image
                         out_channels=10,
                         kernel_size=(5, 5), # kernel is usually a square so a tuple also works
                         stride=2,
                         padding=0)

# Pass single image through new conv_layer_2 (this calls nn.Conv2d()'s forward() method on the input)
conv_layer_2(test_image.unsqueeze(dim=0)).shape

In [ ]:
# Check out the conv_layer_2 internal parameters
print(conv_layer_2.state_dict())

In [ ]:
# Get shapes of weight and bias tensors within conv_layer_2
print(f"conv_layer_2 weight shape: \n{conv_layer_2.weight.shape} -> [out_channels=10, in_channels=3, kernel_size=5, kernel_size=5]")
print(f"\nconv_layer_2 bias shape: \n{conv_layer_2.bias.shape} -> [out_channels=10]")

In [ ]:
# Print out original image shape without and with unsqueezed dimension
print(f"Test image original shape: {test_image.shape}")
print(f"Test image with unsqueezed dimension: {test_image.unsqueeze(dim=0).shape}")

# Create a sample nn.MaxPoo2d() layer
max_pool_layer = nn.MaxPool2d(kernel_size=2)

# Pass data through just the conv_layer
test_image_through_conv = conv_layer(test_image.unsqueeze(dim=0))
print(f"Shape after going through conv_layer(): {test_image_through_conv.shape}")

# Pass data through the max pool layer
test_image_through_conv_and_max_pool = max_pool_layer(test_image_through_conv)
print(f"Shape after going through conv_layer() and max_pool_layer(): {test_image_through_conv_and_max_pool.shape}")

In [ ]:
torch.manual_seed(42)
# Create a random tensor with a similiar number of dimensions to our images
random_tensor = torch.randn(size=(1, 1, 2, 2))
print(f"Random tensor:\n{random_tensor}")
print(f"Random tensor shape: {random_tensor.shape}")

# Create a max pool layer
max_pool_layer = nn.MaxPool2d(kernel_size=2) # see what happens when you change the kernel_size value 

# Pass the random tensor through the max pool layer
max_pool_tensor = max_pool_layer(random_tensor)
print(f"\nMax pool tensor:\n{max_pool_tensor} <- this is the maximum value from random_tensor")
print(f"Max pool tensor shape: {max_pool_tensor.shape}")

In [ ]:
# Setup loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_2.parameters(), 
                             lr=0.1)
try:
    from torchmetrics import Accuracy 
except:
    !pip install torchmetrics
    from torchmetrics import Accuracy

accuracy = Accuracy(task="multiclass",
                    num_classes=len(class_names),
                    )

In [ ]:
torch.manual_seed(42)
from tqdm.auto import tqdm

# Measure time
from timeit import default_timer as timer
train_time_start_model_2 = timer()

# Train and test model 
epochs = 10
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train_step(data_loader=train_dataloader, 
        model=model_2, 
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy=accuracy,
        device=device
    )
    test_step(data_loader=test_dataloader,
        model=model_2,
        loss_fn=loss_fn,
        accuracy=accuracy,
        device=device
    )

train_time_end_model_2 = timer()
total_train_time_model_2 = print_train_time(start=train_time_start_model_2,
                                           end=train_time_end_model_2,
                                           device=device)

In [ ]:
# Get model_2 results 
model_2_results = eval_model(
    model=model_2,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy=accuracy
)
model_2_results

In [ ]:
import pandas as pd
compare_results = pd.DataFrame([model_0_results, model_1_results, model_2_results])
compare_results

In [ ]:
# Add training times to results comparison
compare_results["training_time"] = [total_train_time_model_0,
                                    total_train_time_model_1,
                                    total_train_time_model_2]
compare_results

In [ ]:
# Visualize our model results
compare_results.set_index("model_name")["model_acc"].plot(kind="barh")
plt.xlabel("accuracy (%)")
plt.ylabel("model");

In [ ]:
def make_predictions(model: torch.nn.Module, data: list, device: torch.device = device):

    pred_probs = []
    model.eval()
    with torch.inference_mode():
        for sample in data:

            #Prepare sample 
            sample = torch.unsqueeze(sample, dim=0).to(device)

            # Forward Pass, get raw logit 
            pred_logit = model(sample)

            # Prediction probability 
            pred_prob = torch.softmax(pred_logit.squeeze(), dim=0)

            # Get pred_prob off GPU for further calculations
            pred_probs.append(pred_prob.cpu())
    
    # Stack the pred_probs to turn list into a tensor
    return torch.stack(pred_probs)

In [ ]:
import random
random.seed(42)
test_samples = []
test_labels = []
for sample, label in random.sample(list(test_data), k=9):
    test_samples.append(sample)
    test_labels.append(label)

# View the first test sample shape and label
print(f"Test sample image shape: {test_samples[0].shape}\nTest sample label: {test_labels[0]} ({class_names[test_labels[0]]})")

In [ ]:
len(test_samples)

In [ ]:
# Make predictions on test samples with model 2
pred_probs= make_predictions(model=model_2, 
                             data=test_samples)

# View first two prediction probabilities list
print(torch.round(pred_probs[:2], decimals=3))

In [ ]:
pred_probs.shape

In [ ]:
# Turn the prediction probabilities into prediction labels by taking the argmax()
pred_classes = pred_probs.argmax(dim=1)
pred_classes

In [ ]:
# Are our predictions in the same form as our test labels? 
test_labels, pred_classes

In [ ]:
a = torch.eq(torch.Tensor(test_labels), pred_classes)
print(a)
print(a.sum()/len(a))

In [ ]:
# Plot predictions
plt.figure(figsize=(9, 9))
nrows = 3
ncols = 3
for i, sample in enumerate(test_samples):
  # Create a subplot
  plt.subplot(nrows, ncols, i+1)

  # Plot the target image
  plt.imshow(sample.squeeze(), cmap="gray")

  # Find the prediction label (in text form, e.g. "Sandal")
  pred_label = class_names[pred_classes[i]]

  # Get the truth label (in text form, e.g. "T-shirt")
  truth_label = class_names[test_labels[i]] 

  # Create the title text of the plot
  title_text = f"Pred: {pred_label} | Truth: {truth_label}"
  
  # Check for equality and change title colour accordingly
  if pred_label == truth_label:
      plt.title(title_text, fontsize=10, c="g") # green text if correct
  else:
      plt.title(title_text, fontsize=10, c="r") # red text if wrong
  plt.axis(False);

In [ ]:
# Import tqdm for progress bar
from tqdm.auto import tqdm

# 1. Make predictions with trained model
y_preds = []
model_2.eval()
with torch.inference_mode():
  for X, y in tqdm(test_dataloader, desc="Making predictions"):
    # Send data and targets to target device
    X, y = X.to(device), y.to(device)
    # Do the forward pass
    y_logit = model_2(X)
    # Turn predictions from logits -> prediction probabilities -> predictions labels
    y_pred = torch.softmax(y_logit, dim=1).argmax(dim=1)
    # Put predictions on CPU for evaluation
    y_preds.append(y_pred.cpu())
# Concatenate list of predictions into a tensor
y_pred_tensor = torch.cat(y_preds)

In [ ]:
# See if torchmetrics exists, if not, install it
try:
    import torchmetrics, mlxtend
    print(f"mlxtend version: {mlxtend.__version__}")
    assert int(mlxtend.__version__.split(".")[1]) >= 19, "mlxtend verison should be 0.19.0 or higher"
except:
    !pip install -q torchmetrics -U mlxtend # <- Note: If you're using Google Colab, this may require restarting the runtime
    import torchmetrics, mlxtend
    print(f"mlxtend version: {mlxtend.__version__}")

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix 

#  Setup confusion matrix instance and compare predictions to targets

confmat = ConfusionMatrix(num_classes=len(class_names), 
                          task="multiclass")

confmat_tensor = confmat(preds=y_pred_tensor,
                         target=test_data.targets)

# Plotting 
fig, ax = plot_confusion_matrix(
    conf_mat=confmat_tensor.numpy(), # matplotlib likes working with NumPy 
    class_names=class_names, # turn the row and column labels into class names
    figsize=(10, 7)
);

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(test_data.targets, y_pred_tensor, display_labels=class_names,xticks_rotation="vertical")

#plt.show()

In [ ]:
# Normalized 
ConfusionMatrixDisplay.from_predictions(test_data.targets, 
                                        y_pred_tensor, 
                                        display_labels=class_names,
                                        normalize='true',
                                        xticks_rotation="vertical",
                                        values_format=".0%")


In [ ]:
sample_weight = (test_data.targets != y_pred_tensor)
ConfusionMatrixDisplay.from_predictions(test_data.targets, 
                                        y_pred_tensor,
                                        sample_weight=sample_weight, 
                                        display_labels=class_names,
                                        normalize='true',
                                        xticks_rotation="vertical",
                                        values_format=".0%")
plt.show()


In [ ]:
sample_weight = (test_data.targets != y_pred_tensor)
ConfusionMatrixDisplay.from_predictions(test_data.targets, 
                                        y_pred_tensor,
                                        sample_weight=sample_weight, 
                                        display_labels=class_names,
                                        normalize='pred',
                                        xticks_rotation="vertical",
                                        values_format=".0%")
plt.show()